In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
import itertools

In [8]:
corpus = list()
labels = list()

with open(Path('./output/benign/all_benign.txt'), 'r') as f:
  lines = f.read().split('\n')[:-1]
  # print(lines[0][33:])
for line in lines:
  doc = line[33:]
  corpus.append(doc)
  labels.append(0)

with open(Path('./output/malware/all_malware.txt'), 'r') as f:
  lines = f.read().split('\n')[:-1]
  # print(lines[0][33:])
for line in lines:
  doc = line[33:]
  corpus.append(doc)
  labels.append(1)

In [11]:
y = np.array(labels)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv1 = CountVectorizer()
X1 = cv1.fit_transform(corpus).toarray()
cv2 = CountVectorizer(ngram_range=(2,2))
X2 = cv2.fit_transform(corpus).toarray()
cv3 = CountVectorizer(ngram_range=(3, 3))
X3 = cv3.fit_transform(corpus).toarray()

In [19]:
from sklearn.model_selection import train_test_split

X11_train, X11_test, y11_train, y11_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)
X21_train, X21_test, y21_train, y21_test = train_test_split(X2, y, test_size = 0.2, random_state = 0)
X31_train, X31_test, y31_train, y31_test = train_test_split(X2, y, test_size = 0.2, random_state = 0)

In [20]:
v1=np.array(X1).astype(np.float32)
for i in range(len(corpus)):
    s=sum(X1[i])
    v1[i]=((X1[i]/s)*100).astype(np.float32)
    
X12_train, X12_test, y12_train, y12_test = train_test_split(v1, y, test_size = 0.25, random_state = 0)

v2=np.array(X2).astype(np.float32)
for i in range(len(corpus)):
    s=sum(X2[i])
    v2[i]=((X2[i]/s)*100).astype(np.float32)

X22_train, X22_test, y22_train, y22_test = train_test_split(v2, y, test_size = 0.25, random_state = 0)

v3=np.array(X3).astype(np.float32)
for i in range(len(corpus)):
    s=sum(X3[i])
    v3[i]=((X3[i]/s)*100).astype(np.float32)

X32_train, X32_test, y32_train, y32_test = train_test_split(v3, y, test_size = 0.25, random_state = 0)

In [21]:
print("GB \n")

from sklearn.naive_bayes import GaussianNB
classifierGB11 = GaussianNB()
classifierGB11.fit(X11_train, y11_train)
y11_pred = classifierGB11.predict(X11_test)
cm11 = confusion_matrix(y11_test, y11_pred)
print(cm11)
print(accuracy_score(y11_test, y11_pred))
print('\n')

GB 

[[ 36 248]
 [ 21 255]]
0.5196428571428572




In [22]:
classifierGB12 = GaussianNB()
classifierGB12.fit(X12_train, y12_train)
y12_pred = classifierGB12.predict(X12_test)
cm12 = confusion_matrix(y12_test, y12_pred)
print(cm12)
print(accuracy_score(y12_test, y12_pred))
print('\n')

[[341  12]
 [264  83]]
0.6057142857142858




In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifierrf11 = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf11.fit(X11_train, y11_train)
classifierrf12= RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf12.fit(X12_train, y12_train)
classifierrf21 = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf21.fit(X21_train, y21_train)
classifierrf22 = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf22.fit(X22_train, y22_train)
classifierrf31 = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf31.fit(X31_train, y31_train)
classifierrf32 = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)
classifierrf32.fit(X32_train, y32_train)


yrf11_pred=classifierrf11.predict(X11_test)
yrf12_pred=classifierrf12.predict(X12_test)
yrf21_pred=classifierrf21.predict(X21_test)
yrf22_pred=classifierrf22.predict(X22_test)
yrf31_pred=classifierrf31.predict(X31_test)
yrf32_pred=classifierrf32.predict(X32_test)

In [26]:
cm11 = confusion_matrix(y11_test, yrf11_pred)
print(cm11)
print(metrics.classification_report(y11_test, yrf11_pred, digits=4))

[[271  13]
 [ 21 255]]
              precision    recall  f1-score   support

           0     0.9281    0.9542    0.9410       284
           1     0.9515    0.9239    0.9375       276

    accuracy                         0.9393       560
   macro avg     0.9398    0.9391    0.9392       560
weighted avg     0.9396    0.9393    0.9393       560



In [27]:
cm12 = confusion_matrix(y12_test, yrf12_pred)
print(cm12)
print(metrics.classification_report(y12_test, yrf12_pred,digits=4))
print('\n')

[[340  13]
 [ 32 315]]
              precision    recall  f1-score   support

           0     0.9140    0.9632    0.9379       353
           1     0.9604    0.9078    0.9333       347

    accuracy                         0.9357       700
   macro avg     0.9372    0.9355    0.9356       700
weighted avg     0.9370    0.9357    0.9357       700





In [28]:
cm21 = confusion_matrix(y21_test, yrf21_pred)
print(cm21)
print(metrics.classification_report(y21_test, yrf21_pred, digits=4))
print('\n')

[[275   9]
 [ 23 253]]
              precision    recall  f1-score   support

           0     0.9228    0.9683    0.9450       284
           1     0.9656    0.9167    0.9405       276

    accuracy                         0.9429       560
   macro avg     0.9442    0.9425    0.9428       560
weighted avg     0.9439    0.9429    0.9428       560





In [29]:
cm22 = confusion_matrix(y22_test, yrf22_pred)
print(cm22)
print(metrics.classification_report(y22_test, yrf22_pred, digits=4))
print('\n')

[[344   9]
 [ 32 315]]
              precision    recall  f1-score   support

           0     0.9149    0.9745    0.9438       353
           1     0.9722    0.9078    0.9389       347

    accuracy                         0.9414       700
   macro avg     0.9436    0.9411    0.9413       700
weighted avg     0.9433    0.9414    0.9413       700





In [30]:
cm31 = confusion_matrix(y31_test, yrf31_pred)
print(cm31)
print(metrics.classification_report(y31_test, yrf31_pred, digits=4))
print('\n')

[[275   9]
 [ 23 253]]
              precision    recall  f1-score   support

           0     0.9228    0.9683    0.9450       284
           1     0.9656    0.9167    0.9405       276

    accuracy                         0.9429       560
   macro avg     0.9442    0.9425    0.9428       560
weighted avg     0.9439    0.9429    0.9428       560





In [31]:
cm32 = confusion_matrix(y32_test, yrf32_pred)
print(cm32)
print(metrics.classification_report(y32_test, yrf32_pred, digits=4))
print('\n')

[[346   7]
 [ 40 307]]
              precision    recall  f1-score   support

           0     0.8964    0.9802    0.9364       353
           1     0.9777    0.8847    0.9289       347

    accuracy                         0.9329       700
   macro avg     0.9370    0.9324    0.9326       700
weighted avg     0.9367    0.9329    0.9327       700





In [32]:
print(cm32)
print(metrics.classification_report(y32_test, yrf32_pred, digits=4))
print('\n')

[[346   7]
 [ 40 307]]
              precision    recall  f1-score   support

           0     0.8964    0.9802    0.9364       353
           1     0.9777    0.8847    0.9289       347

    accuracy                         0.9329       700
   macro avg     0.9370    0.9324    0.9326       700
weighted avg     0.9367    0.9329    0.9327       700





In [36]:
from sklearn.svm import SVC

classifiersvc11 = SVC(kernel = 'linear', random_state = 0)
classifiersvc11.fit(X11_train, y11_train)
classifiersvc12= SVC(kernel = 'linear', random_state = 0)
classifiersvc12.fit(X12_train, y12_train)
classifiersvc21 = SVC(kernel = 'linear', random_state = 0)
classifiersvc21.fit(X21_train, y21_train)
classifiersvc22 = SVC(kernel = 'linear', random_state = 0)
classifiersvc22.fit(X22_train, y22_train)

ysvc11_pred=classifiersvc11.predict(X11_test)
ysvc12_pred=classifiersvc12.predict(X12_test)
ysvc21_pred=classifiersvc21.predict(X21_test)
ysvc22_pred=classifiersvc22.predict(X22_test)

In [38]:
cm11 = confusion_matrix(y11_test, ysvc11_pred)
print(cm11)
print(metrics.classification_report(y11_test, ysvc11_pred, digits=4))
print('\n')
cm12 = confusion_matrix(y12_test, ysvc12_pred)
print(cm12)
print('\n')
print(metrics.classification_report(y12_test, ysvc12_pred, digits=4))
cm21 = confusion_matrix(y21_test, ysvc21_pred)
print(cm21)
print('\n')
print(metrics.classification_report(y21_test, ysvc21_pred, digits=4))
cm22 = confusion_matrix(y22_test, ysvc22_pred)
print(cm22)
print(metrics.classification_report(y22_test, ysvc22_pred, digits=4))
print('\n')

[[172 112]
 [103 173]]
              precision    recall  f1-score   support

           0     0.6255    0.6056    0.6154       284
           1     0.6070    0.6268    0.6168       276

    accuracy                         0.6161       560
   macro avg     0.6162    0.6162    0.6161       560
weighted avg     0.6164    0.6161    0.6161       560



[[264  89]
 [115 232]]


              precision    recall  f1-score   support

           0     0.6966    0.7479    0.7213       353
           1     0.7227    0.6686    0.6946       347

    accuracy                         0.7086       700
   macro avg     0.7097    0.7082    0.7080       700
weighted avg     0.7095    0.7086    0.7081       700

[[240  44]
 [ 31 245]]


              precision    recall  f1-score   support

           0     0.8856    0.8451    0.8649       284
           1     0.8478    0.8877    0.8673       276

    accuracy                         0.8661       560
   macro avg     0.8667    0.8664    0.8661       560